In [1]:
import pandas as pd
import requests
import json
import prettytable

In [2]:
#----------------------------------------------- Query for Api ---------------------------------------------------
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SAF11','CUUR0000SEFV03', 'CUUR0000SA0E', 'CUUR0000SAM', 'CUUR0000SEHA','CUUR0000SEHC', 'CUUR0000SEHD', 'CUUR0000SETD', 'CUUR0000SETF', 'CUUR0000SETE', 'CUUR0000SETG02', 'CUUR0000SETG03', 'CUUR0000SEED', 'CUUR0000SEEE04'],"startyear":"2022", "endyear":"2022"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
    
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.csv','w')
    output.write (x.get_string())
    output.close()

In [3]:
#-------------------------------------- Create Index DataFrame -------------------------------------------------
# Item Code Structure, CUUR0000 <- Signals from CPI report || SAF1 <- Exact Item. 
## To Do: Switch from code to series id 
Indexes={'code': ['CUUR0000SAF11', 'CUUR0000SEFV03', 'CUUR0000SA0E', 'CUUR0000SAM', 'CUUR0000SEHA','CUUR0000SEHC', 'CUUR0000SEHD', 'CUUR0000SETD', 'CUUR0000SETF', 'CUUR0000SETE', 'CUUR0000SETG02', 'CUUR0000SETG03', 'CUUR0000SEED', 'CUUR0000SEEE04'], 
        'index': ["Food at Home", "Food at Employee Site & School", "Energy", "Medical Care", "Rent of Primary Residence", "Owner Equiv. Rent", "Tenant Issurance", "Vehicle Maintenance", "Vehicle Fee", "Vehicle Issurance", "Other Intercity Transp", "Other Intracity Transp", "Telephone Services", "Telephone Hardware"],
        'weight': [0.082, 0.042, 0.092, .08237, 0.07195, 0.23502,0.00362,0.01023,0.00493,0.002384, 0.0009, 0.00195, 0.01781, 0.00318]}
Indexes=pd.DataFrame(Indexes)

In [4]:
#------------------------------------- Creating List of CPIs ----------------------------------------------------
fps=['CUUR0000SAF11.csv', 'CUUR0000SEFV03.csv', 'CUUR0000SA0E.csv', 'CUUR0000SAM.csv', 'CUUR0000SEHA.csv','CUUR0000SEHC.csv', 'CUUR0000SEHD.csv', 'CUUR0000SETD.csv', 'CUUR0000SETF.csv', 'CUUR0000SETE.csv', 'CUUR0000SETG02.csv', 'CUUR0000SETG03.csv', 'CUUR0000SEED.csv', 'CUUR0000SEEE04.csv']

In [5]:
#-------------------------------------- Initializing Dataframes ------------------------------------------------
sdata={'year': [], 
        'period': [],
        'value': [],
       'index':[],
       'weight':[]}
sdata=pd.DataFrame(sdata)

In [6]:

for i in range(0,len(fps)):
#--------------------------------------- Reading in Outputed Files --------------------------------------------- 
    df=pd.read_csv(fps[i])
#----------------------------------------- Removing Unwanted Data -------------------------------------------- 
    df.columns = df.iloc[0]
    df = df.reindex(df.index.drop(0)).reset_index(drop=True)
    df.columns.name = None
    df=df.drop(df.index[0])
#----------------------------------------Creating Data Frame----------------------------------------------------------------
## Creating Column Headers
    cols=df.columns
    cols=cols[0]
    cols=cols.split("|")
    for j in range(0,len(cols)):
        cols[j]=cols[j].strip()
## Creating Rows
    rows=[]
    for j in range(0,len(df.index)):
        h=df.iloc[j]
        h=h[0]
        h=h.split("|")
        rows.append(h)
        for k in range(0, len(rows[j])):
            rows[j][k]=rows[j][k].strip()
    rows
## Convert to dataframe 
    CPI=pd.DataFrame(rows, columns=cols)
#------------------------------------------- Final clean of Data -----------------------------------------------
## Filtering Columns
    CPI=CPI[["series id", "year", "period", "value"]]
## Remove unnecessary rows
    CPI=CPI.drop(CPI.index[len(CPI)-1])
## Remove M from period
    CPI["period"]=CPI["period"].str[1:]
## Change Column Types 
    CPI[["year","period","value"]]=CPI[["year", "period", "value"]].apply(pd.to_numeric)
#--------------------------------------- Merging Index values and weights with data   ------------------------- 
    merged=pd.merge(CPI, Indexes, how='left', left_on="series id", right_on="code")
    data2use=merged.drop(columns=["series id", "code"])
    data2use
#--------------------------------------- Appending Value to DataFrame    
    sdata=pd.concat([sdata,data2use])


In [7]:
#--------------------------------------  Creating & Exporting Final Index ------------------------------------------------
## Creating IndexValue Column
sdata['IndexValue']=sdata['weight']*sdata['value']
## Creating New Total Value Added by Index
EssentialCPI=sdata.groupby(by="period").sum()
## Cleaning New Essential CPI
EssentialCPI['year']=2022
EssentialCPI['Index']=EssentialCPI['IndexValue']
EssentialCPI=EssentialCPI.reset_index()
EssentialCPI=EssentialCPI[['period','year', 'Index']]
EssentialCPI['M/M % Ch.'] = (EssentialCPI['Index'] / EssentialCPI['Index'].shift(1) - 1).fillna(0)
## Exporting Final Index Created 
EssentialCPI.to_csv('RefinedEssentialInitial.csv')
EssentialCPI

,period,year,Index,M/M % Ch.
0,1.0,2022,210.961460,0.000000
1,2.0,2022,212.602439,0.007779
2,3.0,2022,216.466657,0.018176
3,4.0,2022,217.435716,0.004477
4,5.0,2022,220.262691,0.013001
5,6.0,2022,224.747907,0.020363
6,7.0,2022,224.710162,-0.000168
7,8.0,2022,225.110536,0.001782
8,9.0,2022,228.131710,0.013421
9,10.0,2022,229.482809,0.005922


In [9]:
EssentialCPI['Annualized']=((1+EssentialCPI['M/M % Ch.'])**12)-1
EssentialCPI

,period,year,Index,M/M % Ch.,Annualized
0,1.0,2022,210.961460,0.000000,0.000000
1,2.0,2022,212.602439,0.007779,0.097442
2,3.0,2022,216.466657,0.018176,0.241290
3,4.0,2022,217.435716,0.004477,0.055063
4,5.0,2022,220.262691,0.013001,0.167672
5,6.0,2022,224.747907,0.020363,0.273669
6,7.0,2022,224.710162,-0.000168,-0.002013
7,8.0,2022,225.110536,0.001782,0.021592
